# Statistical models in R
This notebook is covering:
1. Naïve model

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [3]:
# rpy2 is a Python package that allows you to run R code from Python
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

The magic function `%%R` is used for running R code in Jupyter

In [5]:
%%R
# Install required packages
if (require("dplyr") == FALSE) {
  install.packages("dplyr")
  library(dplyr)
}
if (require("zoo") == FALSE) {
  install.packages("zoo")
  library(zoo)
}
if (require("psych") == FALSE) {
  install.packages("psych")
  library(psych)
}
if (require("TSA") == FALSE) {
  install.packages("TSA")
  library(TSA)
}
if (require("forecast") == FALSE) {
  install.packages("forecast")
  library(forecast)
}
if (require("Metrics") == FALSE) {
  install.packages("Metrics")
  library(Metrics)
}
if (require("ggplot2") == FALSE) {
  install.packages("ggplot2")
  library(ggplot2)
}
if (require("vars") == FALSE) {
  install.packages("vars")
  library(vars)
}
if (require("svars") == FALSE) {
  install.packages("svars")
  library(svars)
}
if (require("tsDyn") == FALSE) {
  install.packages("tsDyn")
  library(tsDyn)
}


Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: psych
Loading required package: TSA

Attaching package: ‘TSA’

The following objects are masked from ‘package:stats’:

    acf, arima

The following object is masked from ‘package:utils’:

    tar

Loading required package: forecast
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Registered S3 methods overwritten by 'forecast':
  method       from
  fitted.Arima TSA 
  plot.Arima   TSA 
Loading required package: Metrics

Attaching package: ‘Metrics’

The following object is masked from ‘package:forecast’:

    accuracy

Loading required package: 

In [6]:
%%R
# Load data
hub_prices <- list(
  nbp = read.csv("../data/interpolated/nbp_close_interpolated.csv"),
  peg = read.csv("../data/interpolated/peg_close_interpolated.csv"),
  the = read.csv("../data/interpolated/the_close_interpolated.csv"),
  ttf = read.csv("../data/interpolated/ttf_close_interpolated.csv"),
  ztp = read.csv("../data/interpolated/ztp_close_interpolated.csv")
)

In [17]:
%%R


hub1_name <- "ttf"


hub1 <- hub_prices[[hub1_name]]
hubs <- data.frame(hub1 = hub1$CLOSE)

In [18]:
%%R
horizon <- 250
window_size <- 5
train_size <- nrow(hubs) - horizon - window_size
hub_train <- hubs[1:train_size+1, ]

In [19]:
%%R
window_size <- 5
horizon <- 250

hubs_lag <- lag(hubs, window_size)


hubs_prediction <- tail(hubs_lag, n = horizon)
hubs_actual <- tail(hubs, n = horizon)


hub1_naive_predictions <- hubs_prediction$hub1
hub1_naive_actuals <- hubs_actual$hub1


hub1_naive_mae <- mae(hub1_naive_actuals, hub1_naive_predictions)

hub1_naive_rmse <- rmse(hub1_naive_actuals, hub1_naive_predictions)
print(paste0(hub1_name, ": Mean Absolute Error: ", hub1_naive_mae))
print(paste0(hub1_name, ": Mean Squared Error: ", hub1_naive_rmse))


[1] "ttf: Mean Absolute Error: 2.13959333333333"
[1] "ttf: Mean Squared Error: 2.96090183926752"


In [20]:
%%R
naive_predictions <- hubs_prediction
naive_actuals <- hubs_actual
colnames(naive_predictions) <- c(hub1_name)
colnames(naive_actuals) <- c(hub1_name)
prediction_dates = tail(hub1$Date, horizon)
naive_predictions <- cbind(data.frame(Date = prediction_dates), naive_predictions)
naive_actuals <- cbind(data.frame(Date = prediction_dates), naive_actuals)

In [21]:
%%R
write.csv(naive_predictions, paste0("../predictions/test/predictions/",hub1_name, "_h", horizon, "_w", window_size, "_naive_predictions.csv"), row.names = FALSE)